In [ ]:

import pandas as pd
import numpy as np
import re
import string
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from scipy.sparse import csr_matrix
import xlrd
import nltk
!pip install nltk==3.5
!pip install regex==2020.6.8
!pip install joblib==0.16.0
!pip install tqdm==4.47.0
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
df =  pd.read_excel('/content/drive/MyDrive/BEPEC/1000 leads.xlsx')


In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  after removing the cwd from sys.path.


In [ ]:
df = df.rename(columns={'Status information':'Status_information','Status ':'Status'})

In [ ]:
df["Location"] = df["Location"].str.lower()

In [ ]:
pattern = r"\d*\/\d*\/[a-zA-z]*:|\d*\/\d*\/\d*\(\w*\):|\d*\|\w*\|\w*:|\(\w*\):|\d*\-\d*\-\d*\(\w*\):|\d*\/\d*\w*:|\d*\/\w*:|\d*\/\d*\/\w*\|:|\d*\/\d*\ *\(\w*\)|\d*\ *\w*\ *\|\ *\w*\ *:|\d*\/\d*\/\d*\(\w*:|\d*\/\w*\/\w*|\d*\/\d*\/\d*\|\(\w*\)|\(\w*\)|\d*\-\d*\-\d*|^\ *|^\:"

In [ ]:
df["Status_information"] = df["Status_information"].apply(lambda x: re.sub(pattern," ",str(x)))

In [ ]:
del df['Unnamed: 4']
del df["Lead Name"]

In [ ]:
df.dropna(inplace=True)

In [ ]:
df['Status'] = df['Status'].str.lower()
df['Status'] = df["Status"].replace(' ','')

df['Status'] = df["Status"].str.replace('conveted','converted')

df['Status'] = df["Status"].str.replace('converted ','converted')

In [ ]:
from sklearn.utils import resample
# Separate majority and minority classes
df_majority = df[df['Status'] == 'not converted']
df_minority = df[df['Status'] == 'converted']
 
# Upsample minority class
df_minority_upsampled = resample(df_minority, 
                                 replace=True,     # sample with replacement
                                 n_samples=len(df_majority),    # to match majority class
                                 random_state=123) # random
 
# Combine majority class with upsampled minority class
df_upsampled = pd.concat([df_majority, df_minority_upsampled])

In [ ]:
def message_text_process(mess):
  no_punctuation = [char for char in mess if char]
  no_punctuation = "".join(no_punctuation)
  return [word for word in no_punctuation.split() if word.lower()
          not in stopwords.words("english")]

In [ ]:
df_upsampled['Status_information'] = df_upsampled['Status_information'].apply(message_text_process).astype(str)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer #bag of words
#data = message_text_process(df_upsampled["Status_information"])
#count_vec = CountVectorizer(analyzer=message_text_process)
count_vec = CountVectorizer()
X = count_vec.fit_transform(df_upsampled["Status_information"])
#X = count_vec.fit_transform(df_upsampled["Status_information"])

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer #TF.IDF
tfidf = TfidfTransformer()
X = tfidf.fit_transform(X)

In [ ]:
X.shape

(1698, 1228)

In [ ]:
X

<1698x1228 sparse matrix of type '<class 'numpy.float64'>'
	with 14190 stored elements in Compressed Sparse Row format>

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, df_upsampled['Status'], test_size=0.2, random_state=42)

In [ ]:
#random foresset classifier with TFIDF
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
Random_Forest_Classifier = RandomForestClassifier()
forest = Random_Forest_Classifier.fit(X_train,y_train)
predicted = forest.predict(X_test)
expected = y_test
print(metrics.classification_report(expected, predicted))
print(metrics.confusion_matrix(expected, predicted))

               precision    recall  f1-score   support

    converted       0.95      0.98      0.97       158
not converted       0.98      0.96      0.97       182

     accuracy                           0.97       340
    macro avg       0.97      0.97      0.97       340
 weighted avg       0.97      0.97      0.97       340

[[155   3]
 [  8 174]]


In [ ]:
import pickle

In [ ]:
pickle_out = open('Random_Forest_Classifier.pkl', 'wb')
pickle.dump(Random_Forest_Classifier, pickle_out)
pickle_out.close()

In [ ]:
pickle_out = open('tfidf_Transformer.pkl', 'wb')
pickle.dump(tfidf, pickle_out)
pickle_out.close()

In [ ]:
pickle_out = open('count_vec.pkl', 'wb')
pickle.dump(count_vec, pickle_out)
pickle_out.close()

In [ ]:
def test_model():
  test_text=input()
  location_label = np.array(29).reshape(-1,1)

  
  bag_words_test = message_text_process(test_text)
  # bag_words =  CountVectorizer(analyzer=message_text_process).fit(test_tex])
  message_bagwords_test = count_vec.transform([test_text])
  #tfid_transformer = TfidfTransformer().fit(message_bagwords)
  message_tfidf = tfidf.transform(message_bagwords_test)
  #message_tfidf = csr_matrix.toarray(message_tfidf)
  #data = np.append(message_tfidf,location_label,axis=1)
  
  y_pred = Random_Forest_Classifier.predict(message_tfidf)

  print(y_pred[0])
  # print(location_label.shape)


In [ ]:
test_model()

i am interested
converted
